In [1]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colorbar import ColorbarBase
import numpy as np
import os
import pandas as pd
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

In [2]:
if not os.path.exists('./Figures'):
    os.makedirs('./Figures')

In [3]:
def add_features1(ax):
    ax.coastlines(resolution="10m")
    ax.add_feature(cfeature.NaturalEarthFeature(category='cultural',
                                                    name='admin_1_states_provinces_lines', 
                                                    scale='50m', facecolor='none'), 
                   edgecolor=[.2,.2,.2])
    ax.add_feature(cfeature.BORDERS, alpha=.5)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.LAND, facecolor=[1,1,.4], alpha=.2)
    ax.set_extent([-124.4, -114, 32.3, 42.3], ccrs.Geodetic())


In [4]:
mask = xr.open_dataset('./Data/mask_CA.nc').mask

proj = ccrs.Orthographic(
    central_longitude=-118,
    central_latitude=37, globe=None)

bounds = [0.05,0.1,0.2,0.3,0.4,0.5, 0.6,0.8,1,1.25,1.5,1.75,2, 2.5,3,3.5,4,5, 6,7,8,10, 
          12,14,16,18,20, 24,28,32,36,50]
c = plt.cm.get_cmap('nipy_spectral_r', len(bounds)+1) # dividing the colormap into specific number of bins
cmaplist = [c(n) for n in range(c.N)]#[1:] #remove the gray color from nipy_spectral
cmap = mpl.colors.ListedColormap([cmaplist[i][:-1] for i in range(len(bounds)+1)])
cmap.set_over(cmaplist[-1][:-1])
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
vmax = bounds[-1]

In [5]:
def plot_pr_hr(ds, ds_sum, xvals, yvals, proj, bounds, cmap, norm, vmax, xtick_labels, closefig):
    fig = plt.figure(figsize=(11, 4.3))
    # Hourly plot:
    ax = fig.add_subplot(1, 4, 1, projection=proj)
    dataplot = ds.pr[0]
    ax.plot = dataplot.where(dataplot>=bounds[0]).plot(
        cmap=cmap, norm=norm, transform=ccrs.PlateCarree(),
        add_colorbar=False,
    )
    add_features1(ax)
    title1 = (pd.to_datetime(ds.time.values[0]) - datetime.timedelta(hours=8)).strftime('%Y-%m-%d %H:%M (PST)')
    plt.title(title1, fontsize=12, color = 'b')
    plt.tight_layout()
    plt.subplots_adjust(top=0.92, bottom=0.20)
    ##########################################
    # Cumulative plot:
    ax = fig.add_subplot(1, 4, 2, projection=proj)
    dataplot = ds_sum.pr.where(dataplot<=bounds[-1],bounds[-1]-.01).where(mask)
    ax.plot = dataplot.where(dataplot>=0.05).plot(
        cmap=cmap, norm=norm, transform=ccrs.PlateCarree(),
        add_colorbar=False,
    )
    add_features1(ax)
    plt.title('Total precip since 2022-12-26', fontsize=12, color = 'b')
    ##########################################
    # Colorbar:
    ax_c = fig.add_axes([0.015, 0.17, 0.477, 0.03])
    cb = ColorbarBase(ax_c, cmap=cmap,  norm=norm, orientation='horizontal', 
                      extend='max', ticks=bounds[:-1])
    cb.ax.set_xticklabels(bounds[:-1])
    cb.ax.tick_params(labelsize=9, rotation=-90)
    cb.set_label(label='Precipitation (inch)', size=12)
    plt.tight_layout()
    plt.subplots_adjust(top=0.92, bottom=0.22)
    ##########################################
    # Timeseries:
    ax = fig.add_axes([0.57, 0.33, 0.41, 0.58])
    x = np.arange(len(xvals))
    plt.plot(x, yvals, color='g', linewidth=2)
    plt.xticks(ticks=np.arange(0,505,24), rotation=-90, fontsize=10)
    ax.set_xticklabels(xtick_labels, fontsize=9)
    plt.ylim([0,30])
    plt.xlim([0,504])
    plt.ylabel('Trillion Gallons', fontsize=11)
    plt.title('Cumulative Precipitation Water across CA', color='b', fontsize=13)
    plt.grid()
    ##########################################
    # Annotation:
    ax.annotate('Visualization: ', xy=(0.55, 0.075), xycoords='figure fraction',
                fontsize=12, fontweight='bold', color='r', ha='left')
    ax.annotate('Ali Ahmadalipour', xy=(0.67, 0.075), xycoords='figure fraction',
                fontsize=12, color='k', ha='left')
    ax.annotate('Data:', xy=(0.55, 0.03), xycoords='figure fraction',
                fontsize=11, fontweight='bold', color='r', ha='left')
    ax.annotate('MRMS (Multi-Radar Multi-Sensor QPE 1hr Pass2), NOAA', xy=(0.598, 0.03), xycoords='figure fraction',
                fontsize=11, color='k', ha='left')


    savename = f'./Figures/Fig_{(pd.to_datetime(ds.time) - datetime.timedelta(hours=8)).strftime("%Y_%m_%d_%H00")[0]}.png'
    plt.savefig(savename, format='png', dpi=200)
    if closefig:
        plt.close()

In [6]:
CA_area_m2 = round(mask.sum().values*0.05*91800*0.05*111111)
inch_to_m = 25.4/1000
m3_to_gallon = 264.172
to_trillion = 1e12

In [7]:
files = glob.glob('./Data/PROCCED_hourly/MRMS_*.nc')
files.sort()
files = files[80:585]

In [8]:
xtick_labels = []
for i in range(0,len(files),24):
    xtick_labels.append(f'{files[i][59:63]}-{files[i][63:65]}-{files[i][65:67]}')

In [9]:
yvals = []
xvals = []

for i in range(len(files)):
    filename = files[i]
    ds = xr.open_dataset(filename)
    ds = ds/25.4 #convert mm to inch
    if i==0:
        ds_sum = ds.mean('time')
    else:
        ds_sum += ds.mean('time')
    xvals.append((pd.to_datetime(ds.time) - datetime.timedelta(hours=8)).values)
    total_rain = float(ds_sum.where(mask).pr.mean().values)
    TrGal = round(total_rain * inch_to_m * CA_area_m2 * m3_to_gallon / to_trillion,2)
    yvals.append(round(TrGal,2))
    # if i > 500:
    #     print(filename)
    #     x
    plot_pr_hr(ds, ds_sum, xvals, yvals, proj, bounds, cmap, norm, vmax, xtick_labels, closefig=True)

## Video

In [10]:
import cv2
import glob

In [11]:
files = glob.glob('./Figures/Fig_*.png')
files.sort()

In [12]:
img_array = []
for filename in files:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('Vid_Prec_CAflood_17fps.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 17, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
